In [2]:
import os

import torch
from torch import nn
from torch import optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
from nflows.flows.base import Flow
from nflows.distributions.normal import StandardNormal
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation

import tqdm as tqdm
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [4]:
def make_checkpoint(flow, optimizer, loss, filename):
    torch.save({'model_state_dict': flow.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
               }, 
               filename)

def make_flow(d, num_layers = 5):
    base_dist = StandardNormal(shape=[d])
    #base_dist = BoxUniform(low = -2*torch.ones(d), high = 2*torch.ones(d))
    
    transforms = []
    for _ in range(num_layers):
        transforms.append(ReversePermutation(features=d))
        transforms.append(MaskedAffineAutoregressiveTransform(features=d, 
                                                              hidden_features=8))
    transform = CompositeTransform(transforms)
    
    return Flow(transform, base_dist)

def train_flow(flow, optimizer, data, dir, num_iter = 10000):
    losses = np.zeros(num_iter)
    print(-flow.log_prob(inputs=data).mean())
    for i in tqdm.trange(num_iter):
        optimizer.zero_grad()
        loss = -flow.log_prob(inputs=data).mean()
        losses[i] = loss
        
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            np.save(dir + 'losses.npy', losses)
            make_checkpoint(flow, optimizer, loss, dir + 'ckpt_{}'.format(i))
            
    make_checkpoint(flow, optimizer, loss, dir + 'ckpt_{}'.format(num_iter))
    np.save(dir + 'losses.npy', losses)

In [5]:
def marginal_X(X, log_prob):
    X = X.repeat_interleave(101).reshape(-1, 1)
    
    Y = torch.linspace(-0.5, 1.5, 101).reshape(-1, 1)
    Y = Y.repeat(n, 1)
    Y = Y.to(device)

    XY = torch.cat([X, Y], dim = 1)
    LP = flow_XY.log_prob(XY)
    PX = []
    for i in range(n):
        PX += [torch.trapezoid(torch.exp(LP[101 * i : 101 * (i + 1)]), dx = 0.02)]
    return torch.stack(PX, dim = 0)

def marginal_Y(Y, log_prob):
    Y = Y.repeat_interleave(101).reshape(-1, 1)
    
    X = torch.linspace(-0.5, 1.5, 101).reshape(-1, 1)
    X = X.repeat(n, 1)
    X = X.to(device)

    XY = torch.cat([X, Y], dim = 1)
    LP = flow_XY.log_prob(XY)
    PY = []
    for i in range(n):
        PY += [torch.trapezoid(torch.exp(LP[101 * i : 101 * (i + 1)]), dx = 0.02)]
    return torch.stack(PY, dim = 0)

In [6]:
np.random.seed(666)

n = 500
r = 0.1

C = stats.multivariate_normal.rvs(np.zeros(2), [[1, r],[r, 1]], size = n)
D = stats.norm.cdf(C)
X = D[:, 0].reshape(n, 1)
Y = D[:, 1].reshape(n, 1)

D = torch.tensor(D, dtype = torch.float32).cuda()
X = torch.tensor(X, dtype = torch.float32).cuda()
Y = torch.tensor(Y, dtype = torch.float32).cuda()
D.to(device)
X.to(device)
Y.to(device)

tensor([[3.8619e-01],
        [3.9725e-01],
        [6.3704e-01],
        [2.5875e-01],
        [1.5802e-01],
        [1.4094e-01],
        [1.1127e-01],
        [4.6517e-01],
        [6.6321e-01],
        [9.1951e-01],
        [9.6921e-01],
        [6.4302e-01],
        [7.6879e-01],
        [6.6828e-02],
        [6.3746e-01],
        [8.3659e-01],
        [2.9554e-01],
        [3.3371e-01],
        [4.6101e-01],
        [2.8297e-01],
        [8.2561e-01],
        [5.4455e-01],
        [4.2983e-01],
        [5.1402e-01],
        [9.0103e-01],
        [8.7779e-01],
        [4.6009e-01],
        [5.4229e-01],
        [2.3262e-01],
        [9.2693e-01],
        [1.0000e+00],
        [2.9643e-01],
        [4.1781e-01],
        [8.2354e-01],
        [6.5676e-01],
        [6.4206e-01],
        [6.8611e-01],
        [9.4694e-01],
        [2.6039e-01],
        [4.4483e-01],
        [7.4748e-01],
        [5.3354e-01],
        [6.1234e-01],
        [6.6576e-01],
        [9.0203e-01],
        [8

In [6]:
flow_XY = make_flow(d = 2)
flow_XY.to(device)
optimizer_XY = optim.Adam(flow_XY.parameters())

os.makedirs('XY', exist_ok = True)
train_flow(flow_XY, optimizer_XY, D, 'XY/')

tensor(5.0370, device='cuda:0', grad_fn=<NegBackward0>)


100%|██████████| 10000/10000 [01:51<00:00, 90.01it/s]


In [7]:
flow_XY.eval();

# Observed Coefficient

In [13]:
LPX = torch.log(marginal_X(X, flow_XY.log_prob))
LPY = torch.log(marginal_Y(Y, flow_XY.log_prob))
ic_obs = flow_XY.log_prob(D).mean() - LPX.mean() - LPY.mean() 
ic_obs

tensor(0.2131, device='cuda:0', grad_fn=<SubBackward0>)

# $p$ Value

In [42]:
np.linspace(-0.5, 1.5, 51)

array([-0.5 , -0.46, -0.42, -0.38, -0.34, -0.3 , -0.26, -0.22, -0.18,
       -0.14, -0.1 , -0.06, -0.02,  0.02,  0.06,  0.1 ,  0.14,  0.18,
        0.22,  0.26,  0.3 ,  0.34,  0.38,  0.42,  0.46,  0.5 ,  0.54,
        0.58,  0.62,  0.66,  0.7 ,  0.74,  0.78,  0.82,  0.86,  0.9 ,
        0.94,  0.98,  1.02,  1.06,  1.1 ,  1.14,  1.18,  1.22,  1.26,
        1.3 ,  1.34,  1.38,  1.42,  1.46,  1.5 ])

In [7]:
X_idx = np.random.choice(n, size = n, replace = False)
#Y_idx = np.random.choice(n, size = n, replace = False)
X_star = D[X_idx, 0].reshape(-1, 1)
Y_star = Y
D_star = torch.cat([X_star, Y_star], dim = 1)

In [22]:
flow_0 = make_flow(d = 2)
flow_0.to(device)
optimizer_0 = optim.Adam(flow_0.parameters())

os.makedirs('0', exist_ok = True)
train_flow(flow_0, optimizer_0, D_star, '0/')

tensor(5.6255, device='cuda:0', grad_fn=<NegBackward0>)


100%|██████████| 10000/10000 [01:58<00:00, 84.37it/s]


In [23]:
LPX = torch.log(marginal_X(X_star, flow_0.log_prob))
LPY = torch.log(marginal_Y(Y_star, flow_0.log_prob))
ic = flow_0.log_prob(D_star).mean() - LPX.mean() - LPY.mean()
ic

tensor(0.0579, device='cuda:0', grad_fn=<SubBackward0>)

In [43]:
np.load('uniforms/n/1000/s_p_val.npy')

array(0.166)

In [ ]:
ns = 100, 1000, 5000, 10000

In [12]:
reps = 100
info_coeffs = np.zeros(reps)

for i in range(reps):
    idx = np.random.choice(n, size = n)
    D_star = D[idx]
    X_star = D[:, 0].reshape(n, 1)
    Y_star = D[:, 1].reshape(n, 1)

    LPX = torch.log(marginal_X(X_star, flow_XY.log_prob))
    LPY = torch.log(marginal_Y(Y_star, flow_XY.log_prob))

    info_coeffs[i] = LPX.mean() + LPY.mean() - flow_XY.log_prob(D_star).mean()
    print(info_coeffs[i])

0.0014953669160604477
0.000575011596083641
0.000916656106710434
-0.0003197118639945984
-0.0015476178377866745
0.0010863188654184341
0.0038277730345726013
-0.0017951875925064087
-0.0019401367753744125
0.0039041507989168167
-0.005479016341269016
-0.0002898033708333969
0.0015900637954473495
0.0023582447320222855
-0.0026245340704917908
-0.0024624858051538467
0.006149407476186752
-0.0028465110808610916
-0.004073707386851311
-0.000199880450963974
-0.0008739735931158066
-0.0010651741176843643
0.0007824469357728958
0.0012500360608100891
0.002895694226026535
0.0011813677847385406
0.002385195344686508
-0.0019622091203927994
0.002053685486316681
0.002970278263092041
-0.002182161435484886
0.0006743595004081726
-0.0006881691515445709
-0.002055548131465912
-0.0016658101230859756
-0.002171950414776802
-0.002356473356485367
0.001043420284986496
-0.0024791285395622253
-0.0010428093373775482
7.878057658672333e-05
-0.0023751407861709595
0.0034232214093208313
-0.000362444669008255
-0.001771220937371254
-0

KeyboardInterrupt: 

In [16]:
info_coeffs = info_coeffs[info_coeffs != 0]

In [21]:
np.mean(abs(info_coeffs) >= abs(obs.cpu().detach().numpy()))

0.8888888888888888